# Task 3: Prediction with Machine Learning 

### Table of Contents

1. [Introduction](#introduction)
2. [Data Preprocessing](#data-preprocessing)
3. [Training Machine Learning Models](#machine-learning)
    - [Feature Selection](#feature-selection)
3. [Summary](#summary)

    

## Introduction <a class="anchor" id="introduction"></a>

Machine learning will be used to predict house price with the Ames Housing dataset, which has been cleaning during data wrangling and exploratory data analysis (EDA). The first step of this project is to preprocess data based on the insights discovered during exploratory data analysis. Subsequently, the preprocessed data will be used to train various machine learning models such as neural network, XGBoost, linear regression and etc in order to predict house price.

## Data Preprocessing <a class="anchor" id="data-preprocessing"></a>

To preprocess the data, the following steps will be carried out in a pipeline.

1. Split data into k-folds for k-folds cross validation
2. For each fold,
    1. Use ordinal encoding to encode ordinal variables
    2. Use one-hot encoding to encode nominal variables with small cardinality and that seem to have substantial impact of house price (in order to retain all the information)
    3. Use hash encoding to encode nominal variables with high cardinality
    4. Perform standardization
    5. Use k-nearest neighbors (KNN) imputer to impute missing values

In [1]:
import numpy as np 
import pandas as pd
import os
import pickle
from sklearn.model_selection import KFold, ParameterSampler, ParameterGrid
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
import category_encoders as ce
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from scipy.stats import uniform
import xgboost

In [ ]:
class myOrdinalEncoder(BaseEstimator, TransformerMixin):
    """
    Encode ordinal features, which may contain NaNs, as a 2D array
        
    Parameters
    ----------      
    unknown_value: int, default=0
        Unknown value to use for unknown categorical feature which is not seen in training data
    """
    
    
    def __init__( self, unknown_value=0):  
        self.ordinal_encoder = None
        self.categories = []
        self.nan_replacement = None
        self.unknown_value = unknown_value

        
    def fit( self, X, y = None ):
        """
        Fit myOrdinalEncoder to X.
    
        Parameters
        ----------
        X : DataFrame, shape [n_samples, n_features]
            The data to determine the categories of each feature.
        y : None
            Ignored. This parameter exists only for compatibility with
            :class:`~sklearn.pipeline.Pipeline`.
        Returns
        -------
        self
        """      
        # Create a copy of X
        X_copy = X.copy()
        
        # Replace NaNs with replacement before fitting to avoid errors
        self.nan_replacement = X_copy.mode(dropna=True).iloc[0, :]
        X_copy = X_copy.fillna(self.nan_replacement)
        
        for col in X_copy:
            self.categories.append(list(X_copy[col].cat.categories))

        # Fit ordinal encoder
        self.ordinal_encoder = OrdinalEncoder(categories=self.categories, handle_unknown='use_encoded_value', unknown_value=np.nan)
        self.ordinal_encoder.fit(X_copy);
        
        return self

    def transform(self, X): 
        """
        Transform X using ordinal encoding.
        
        Parameters
        ----------
        X : DataFrame, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : 2D array
            Transformed input.
        """
        
        # Create a copy of X
        X_copy = X.copy()
          
        # Create a numpy array that defines the locations of NaNs in the dataframe after hash encoding
        nan_location_arr = X_copy.to_numpy()
        nan_location_arr = pd.isnull(nan_location_arr)

        # Replace NaNs with replacement before fitting to avoid errors
        X_out = X_copy.fillna(self.nan_replacement)
        
        # Transform each feature
        X_out = self.ordinal_encoder.transform(X_out)
        
        # Reconvert back values into NaNs
        X_out[nan_location_arr] = np.nan
        
        return X_out
        
#         X_out = self.ordinal_encoder.transform(X_copy)
    
        return X_copy
    
    
class myOneHotEncoder(BaseEstimator, TransformerMixin):
    """
    Encode categorical features, which may contain NaNs, as a one-hot numeric array
    """
    def __init__(self):
        # Initialize one-hot encoder
        self.one_hot_encoder = OneHotEncoder(drop='first', sparse=False)
        self.nan_replacement = None
        self.one_hot_length = []

    def fit(self, X, y = None ):
        """
        Fit myOneHotEncoder to X.
    
        Parameters
        ----------
        X : DataFrame, shape [n_samples, n_features]
            The data to train the encoder
        y : None
            Ignored. This parameter exists only for compatibility with
            :class:`~sklearn.pipeline.Pipeline`.
        Returns
        -------
        self
        """
        
        # Create a copy of X
        X_copy = X.copy()
        
        # Replace NaNs with replacement before fitting to avoid errors
        self.nan_replacement = X_copy.mode(dropna=True).iloc[0, :]
        X_copy = X_copy.fillna(self.nan_replacement)
        
        # Fit one-hot encoder
        self.one_hot_encoder.fit(X_copy)
        
        # Get the length of one-hot encoding for each feature
        for category in self.one_hot_encoder.categories_:
            # Minus length with 1 because of drop='first' in OneHotEncoder
            self.one_hot_length.append(len(category) - 1) 
            
        return self

    
    def transform(self, X):
        """
        Transform X using one-hot encoding.
        
        Parameters
        ----------
        X : DataFrame, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : 2D array
            Transformed input.
        """
        
        # Create a copy of X
        X_copy = X.copy()
        
        # Replace unknown value with NaN
        for i, category in enumerate(self.one_hot_encoder.categories_):
            X_copy.loc[~X_copy.loc[:, X_copy.columns[i]].isin(category),  X_copy.columns[i]] = np.nan

        # Create a numpy array that defines the locations of NaNs in the dataframe after one-hot encoding
        nan_location_arr = X_copy.to_numpy()
        nan_location_arr = np.repeat(pd.isnull(nan_location_arr), repeats=self.one_hot_length, axis=1)
        
        # Replace NaNs with replacement before fitting to avoid errors
        X_out = X_copy.fillna(self.nan_replacement)
        
        # Transform each feature
        X_out = self.one_hot_encoder.transform(X_out)
        
        # Reconvert back values into NaNs
        X_out[nan_location_arr] = np.nan
        
        return X_out
    
    
class myHashingEncoder(BaseEstimator, TransformerMixin):
    """
    Encode nominal features, which may contain NaNs, as a 2D array
        
    Parameters
    ----------
    n_bits: int, default=8
        Number of bits used to represent each nominal feature
    """
    
    
    def __init__(self, n_bits=8):
        self.n_bits = n_bits
        
        # Initialize hashing encoder
        self.hashing_encoder = ce.hashing.HashingEncoder(return_df=False, n_components=n_bits)
        

    def fit(self, X, y = None ):
        """
        Ignored. This exists only for compatibility with 
            :class:`~sklearn.pipeline.Pipeline`.
        Returns
        -------
        self
        """
        
        return self
    

    def transform(self, X):
        """
        Transform X using hash encoding.
        
        Parameters
        ----------
        X : DataFrame, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : 2D array
            Transformed input.
        """
        
        # Create a numpy array that defines the locations of NaNs in the dataframe after hash encoding
        nan_location_arr = X.to_numpy()
        nan_location_arr = np.repeat(nan_location_arr, repeats=self.n_bits, axis=1)
        
        X_out = np.empty((X.shape[0],0))
        
        for col in X:
            # Convert the data type of column into str if int or float
            if(X[col].dtype == np.float64 or X[col].dtype == np.int64):
                X[col] = X[col].astype(str)
                
            # Transform each feature and convert the data type into float for NaN
            X_out = np.concatenate((X_out, self.hashing_encoder.fit_transform(X[col].to_numpy()).astype('float')), axis=1)
            
        # As hashing encoder will turn NaN into numeric array, reconvert back these values into NaNs
        X_out[pd.isnull(nan_location_arr)] = np.nan
        
        return X_out
    
    
class hashAggregator(BaseEstimator, TransformerMixin):
    """
    Combine all the 2D arrays of encoded using hash encoding after KNN imputation 
        
    Parameters
    ----------
    n_feas: int
        Number of features encoded using hash encoding 
    n_bits: int, default=8
        Number of bits used to represent each nominal feature
    """
    
    
    def __init__(self, n_feas, n_bits):
        self.n_feas = n_feas
        self.n_bits = n_bits
        
        
    def fit(self, X, y = None ):
        """
        Ignored. This exists only for compatibility with 
            :class:`~sklearn.pipeline.Pipeline`.
        Returns
        -------
        self
        """
        
        return self
    

    def transform(self, X):
        """
        Combine all the 2D arrays of features encoded using hash encoding
        
        Parameters
        ----------
        X : array, shape [n_samples, n_features*n_bits]
            The data to combine.
        Returns
        -------
        X_out : 2D array
            Combined input.
        """
        X_copy = np.copy(X)

        # Split input array into two array: one containing data encoded using hash encoding, another containing the rest
        X_copy_non_hash, X_copy_hash = np.split(X_copy, [-self.n_feas*self.n_bits,], axis=1)
        
        # Combine all the 2D arrays of features encoded using hash encoding
        X_copy_hash = X_copy_hash.reshape(X_copy_hash.shape[0], -1, self.n_bits).sum(1)
        
        # Concatenate hash and non-hash arrays
        X_out = np.concatenate((X_copy_non_hash, X_copy_hash), axis=1)
        
        return X_out

In [ ]:
def kfolds_split(df, n_folds):
    # Split data into K-folds
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=0)

    kfold_df = {}

    for i, (train_index, val_index) in enumerate(kf.split(df)):
        kfold_df['Fold'+ str(i)] = df.loc[train_index, :].copy(), df.loc[val_index, :].copy()
        
    return kfold_df
        
        
def data_preprocessing(df, data_types, ordinal_var, n_bits=8):
    
    kfold_data = {}
    
    for key_fold, (df_train, df_val) in df.items():
         
        data_types_per_fold = data_types
        ordinal_var_per_fold = ordinal_var
        
        for key in ordinal_var_per_fold:
            # Remove categorical values from ordinal_var_per_fold that do not exist in training set to avoid data leakage
            train_unique = df_train[key].unique()
            ordinal_var_per_fold[key] = [cat_value for cat_value in ordinal_var_per_fold[key] if cat_value in train_unique]
    
            # Convert variables in training set into ordered categorical types
            ordered_var = pd.api.types.CategoricalDtype(ordered = True, categories = ordinal_var_per_fold[key])
            df_train.loc[:, key] = df_train.loc[:, key].astype(ordered_var)
            
        # Break nominal variables into two group: one-hot encoding or hash encoding
        data_types_per_fold['nom_one_hot'] = []
        data_types_per_fold['nom_hash'] = []

        for var in data_types_per_fold['nom']:
    
            # Based on EDA conducted previously, Neighborhood, MSZoning, MasVnrType and Foundation seems to have 
            # substantial impact on SalePrice. Thus, one-hot encoding will be chosen over hash encoding for these
            # variables, regardless of cardinality, because hash encoding will cause loss in information
            if var in ['Neighborhood', 'MSZoning', 'MasVnrType', 'Foundation']:
                data_types_per_fold['nom_one_hot'].append(var)
        
            else:
                if len(df_train[var].unique()) <= 5 :
                    # use one-hot encoding if the cardinality is small
                    data_types_per_fold['nom_one_hot'].append(var)
                else:
                    # use one-hot encoding if the cardinality is big
                    data_types_per_fold['nom_hash'].append(var) 
                    
        # Define ColumnTransformer to transform columns with different methods
        column_transformer = ColumnTransformer(transformers=[('dis_transformer', 'passthrough', data_types_per_fold['dis']),
                                                             ('con_transformer', 'passthrough', data_types_per_fold['con']),
                                                             ('ord_transformer', myOrdinalEncoder(), data_types_per_fold['ord']),
                                                             ('nom_one_hot_transformer', myOneHotEncoder(), data_types_per_fold['nom_one_hot']),
                                                             ('nom_hash_transformer', myHashingEncoder(n_bits = n_bits), data_types_per_fold['nom_hash']),
                                                            ])


        # Define pipeline for pre-processing data
        preprocessor = Pipeline(steps=[('column_transformer', column_transformer),
                                       ('standard_scaler', StandardScaler()),
                                       ('imputer', KNNImputer(n_neighbors=5, weights='distance')),
                                       ('hash_aggregator', hashAggregator(n_feas=len(data_types_per_fold['nom_hash']), n_bits=n_bits)),
                                      ])

        # Preprocess training and validation data
        X_train = preprocessor.fit_transform(df_train)
        X_val = preprocessor.transform(df_val)
        y_train = df_train['SalePrice'].to_numpy()
        y_val = df_val['SalePrice'].to_numpy()
        
        kfold_data[key_fold] = ((X_train, y_train), (X_val, y_val))
        
    return kfold_data


def data_preparation(filepath, n_folds=5, unwanted_vars_dict=None):  
    """
    Split data into k-folds and pre-process them
        
    Parameters
    ----------
    filepath: string
        Path to file storing data
    n_folds: int
        Number of folds for cross-validation
    unwanted_vars_dict: dict
        Unwanted variables among discrete, continuous, nominal and ordinal variables
    """
    
    # Load training and testing sets
    df = pd.read_csv(filepath)

    # Drop ID column
    df.drop(['Id'], axis=1, inplace=True)

    # Define discrete(dis), continuous(con), nominal(nom), ordinal(ord) variables exlcuding target variable, which is SalePrice
    data_types_dict = {'nom': ['MSSubClass', 'MSZoning', 'Street', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 
                               'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
                               'Heating', 'CentralAir', 'Electrical', 'GarageType', 'PavedDrive', 'SaleType', 'SaleCondition'],
                       'ord': ['LotShape', 'LandContour', 'LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 
                               'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 
                               'GarageFinish', 'GarageQual', 'GarageCond'],
                       'dis': ['YearBuilt', 'YearRemodAdd', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 
                               'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'MoSold', 'YrSold'],
                       'con': ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', 
                               '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 
                               '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']
                      }
    
    # Remove unwanted variables if there is any
    if unwanted_vars_dict is not None:
        for data_type in unwanted_vars_dict.keys():
            data_types_dict[data_type] = list(set(data_types_dict[data_type]) - set(unwanted_vars_dict[data_type]))
    
    # Define order of categorical values in each ordinal variables
    ordinal_var_dict = {'LotShape': ['IR3', 'IR2', 'IR1', 'Reg'],
                        'LandContour': ['Lvl', 'Bnk', 'HLS', 'Low'],
                        'LandSlope': ['Sev', 'Mod', 'Gtl'],
                        'OverallQual': list(range(0,11)),
                        'OverallCond': list(range(0,11)),
                        'ExterQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                        'ExterCond': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                        'BsmtQual': ['NoBsmt', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                        'BsmtCond': ['NoBsmt', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                        'BsmtExposure': ['NoBsmt', 'No', 'Mn', 'Av', 'Gd'],
                        'BsmtFinType1': ['NoBsmt', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
                        'BsmtFinType2': ['NoBsmt', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
                        'HeatingQC': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                        'KitchenQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                        'Functional': ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'],
                        'GarageFinish': ['NoGarage', 'Unf', 'RFn', 'Fin'],
                        'GarageQual': ['NoGarage', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                        'GarageCond': ['NoGarage', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                       }
    
    # Remove ordinal variables from ordinal_var_dict
    if unwanted_vars_dict is not None and unwanted_vars_dict.get('ord') is not None:
        for unwanted_nom_var in unwanted_vars_dict.get('ord'):
            ordinal_var_dict.pop(unwanted_nom_var)
            
    # Split data into n folds
    kfolds_data = kfolds_split(df, n_folds)
    
    # Preprocess data
    kfolds_data = data_preprocessing(kfolds_data, data_types_dict, ordinal_var_dict)
    
    return kfolds_data

## Training Machine Learning Models <a class="anchor" id="machine-learning"></a>

Various machine learning models are used to predict SalePrice. The evaluation metric used for evaluating machine learning models with k-folds cross validation is mean squared log error. Log error is chosen because there is a big variation in values of SalePrice.

In [2]:
# Define URL of training set
dirname = '/kaggle/input'
subdirname = 'dataset'
train_filename = 'train_clean_EDA.csv'
train_filepath = os.path.join(dirname, subdirname, train_filename)

# Define number of folds for cross validation
n_folds = 5

# Define URL of preprocessed data file 
dirname = '/kaggle/input/dataset'
k_folds_data_filename = 'k_folds_data.pkl'
k_folds_data_filepath = os.path.join(dirname, k_folds_data_filename)

if os.path.exists(k_folds_data_filepath):
    # If file exists, then load data from the file
    with open(k_folds_data_filepath, 'rb') as f:
        k_folds_data = pickle.load(f)
else:
    # Else, store the data in a file to avoid re-computation in the future
    with open(os.path.join('/kaggle/working', k_folds_data_filename ), 'wb') as f:
        k_folds_data = data_preparation(train_filepath, n_folds)
        pickle.dump(k_folds_data, f)

In [3]:
def train_model(k_folds_data, models):
    
    if not isinstance(models, list):
        models = [models]
       
    # Empty dataframe, to be used for storing cross-validation performance later
    cv_performance = pd.DataFrame()
    
    best_model_performance = None
    best_model_index = None
    
    for i, model in enumerate(models):
        
        # Empty list, to be used for storing mean squared log error (MSLE)
        msles_train = []
        msles_val = []
        
        for _, ((X_train, y_train), (X_val, y_val)) in k_folds_data.items():
            # Define end-to-end machine learning pipeline
            ml_model = TransformedTargetRegressor(regressor=model, func=np.log, inverse_func=np.exp)
            
            # Train model
            ml_model.fit(X_train, y_train)
            
            # Predict with training and validation data
            y_train_pred = ml_model.predict(X_train)
            y_val_pred = ml_model.predict(X_val)
            
            # Compute root mean squared log error (MSLE) for training data
            msles_train.append(np.round(np.sqrt(mean_squared_log_error(y_train, y_train_pred)), 6))
            
            # Compute root mean squared log error (MSLE) for validation data
            msles_val.append(np.round(np.sqrt(mean_squared_log_error(y_val, y_val_pred)), 6))
            
        msles_train_mean = np.mean(msles_train)
        msles_val_mean = np.mean(msles_val)
        
        # Compare performances of current model and the best model
        if i == 0 or (msles_val_mean < best_model_performance):
            best_model_performance = msles_val_mean
            best_model_index = i+1
        
        # Compute mean of MLSEs after k-folds cross validation
        msles_train.append(msles_train_mean)
        msles_val.append(msles_val_mean)
        
        # Store MLSEs in the dataframe
        cv_performance[str(model) + "_train"] = msles_train
        cv_performance[str(model) + "_val"] = msles_val
        
        print('Finished training {}/{} models'.format(i, len(models)))

    # Update index of the dataframe
    cv_performance.index = ["Fold " + str(i) for i in range(len(k_folds_data))] + ['Mean']
    
    # Return best_model_index and a dataframe containing the training and validation performance of the best model
    return best_model_index, cv_performance[cv_performance.columns[[best_model_index*2-2, best_model_index*2-1]]]

### Ridge Regression

In [4]:
# Parameter grid for ridge regression model
param_grid = {'alpha':[0, 1e-1, 3e-1, 1, 3, 10, 30, 100, 300, 1000]}
param_list = list(ParameterGrid(param_grid))

# Train ridge regression model
models = [Ridge(alpha=val) for param in param_list for _, val in param.items() ]
best_model_index, df_performance = train_model(k_folds_data, models)
df_performance

Finished training 0/10 models
Finished training 1/10 models
Finished training 2/10 models
Finished training 3/10 models
Finished training 4/10 models
Finished training 5/10 models
Finished training 6/10 models
Finished training 7/10 models
Finished training 8/10 models
Finished training 9/10 models


,Ridge(alpha=30)_train,Ridge(alpha=30)_val
Fold 0,0.099624,0.126681
Fold 1,0.105371,0.104219
Fold 2,0.100818,0.131173
Fold 3,0.101336,0.120066
Fold 4,0.106405,0.099264
Mean,0.102711,0.116281


### Lasso Regression

In [ ]:
# Parameter grid for lasso regression model
param_grid = {'alpha':[0, 1e-5, 3e-5, 1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2, 1e-1, 3e-1, 1, 3]}
param_list = list(ParameterGrid(param_grid))

# Train lasso regression model
models = [Lasso(alpha=val, max_iter=1e7) for param in param_list for _, val in param.items() ]
best_model_index, df_performance = train_model(k_folds_data, models)
df_performance

### Elastic Net Regression

In [ ]:
# Parameter grid for elastic net regression model
param_grid = {'alpha':[1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2, 1e-1, 3e-1, 1],             
              'l1_ratio':[0, 1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2, 1e-1, 3e-1, 1]}
param_list = list(ParameterSampler(param_grid, n_iter=30))

# Train elastic net regression model
models = []
for param in param_list:
    models.append(ElasticNet(alpha=param['alpha'], l1_ratio=param['l1_ratio'], max_iter=1e7))
best_model_index, df_performance = train_model(k_folds_data, models)
df_performance

### eXtreme Gradient Boosting (XGBoost)

In [ ]:
def train_xgboost_model(k_folds_data, models):
    
    if not isinstance(models, list):
        models = [models]
       
    # Empty dataframe, to be used for storing cross-validation performance later
    cv_performance = pd.DataFrame()
    
    best_model_performance = None
    best_model = None
    best_model_index = None
    
    for i, model in enumerate(models):
        
        # Empty list, to be used for storing mean squared log error (MSLE)
        msles_train = []
        msles_val = []
        
        for _, ((X_train, y_train), (X_val, y_val)) in k_folds_data.items():
            # Define end-to-end machine learning pipeline
            ml_model = TransformedTargetRegressor(regressor=model, func=np.log, inverse_func=np.exp)
            
            # Train model
            ml_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric=['rmsle'], early_stopping_rounds=10, verbose=False)
            
            # Predict with training and validation data
            y_train_pred = ml_model.predict(X_train)
            y_val_pred = ml_model.predict(X_val)
            
            # Compute mean squared log error (MSLE) for training data
            msles_train.append(np.round(np.sqrt(mean_squared_log_error(y_train, y_train_pred)), 6))
            
            # Compute mean squared log error (MSLE) for validation data
            msles_val.append(np.round(np.sqrt(mean_squared_log_error(y_val, y_val_pred)), 6))
            
        msles_train_mean = np.mean(msles_train)
        msles_val_mean = np.mean(msles_val)
        
        if i == 0 or (msles_val_mean < best_model_performance):
            best_model_performance = msles_val_mean
            best_model_index = i+1
            best_model = ml_model
        
        # Compute mean of MLSEs after k-folds cross validation
        msles_train.append(msles_train_mean)
        msles_val.append(msles_val_mean)
       
        # Store MLSEs in the dataframe
        cv_performance[str(model) + "_train"] = msles_train
        cv_performance[str(model) + "_val"] = msles_val
        
        print('Finished training {}/{} models'.format(i, len(models)))

    # Update index of the dataframe
    cv_performance.index = ["Fold " + str(i) for i in range(len(k_folds_data))] + ['Average']
    
    return best_model, best_model_index, cv_performance[cv_performance.columns[[best_model_index*2-2, best_model_index*2-1]]]

In [ ]:
# Parameter grid for XGBoost
param_grid = {'n_estimator':list(range(1500, 3500, 20)), 
              'max_depth':[3, 4, 5, 6, 7],
              'learning_rate':[0.001, 0.003, 0.01, 0.03, 0.1, 0.3],
              'reg_alpha':uniform(),
              'reg_lambda':uniform(),
             }
param_list = list(ParameterSampler(param_grid, n_iter=200))


# Train XGBoost model
models = []
for param in param_list:
    models.append(xgboost.XGBRegressor(n_estimators=param['n_estimator'], 
                                       max_depth=param['max_depth'], 
                                       learning_rate=param['learning_rate'], 
                                       reg_alpha=param['reg_alpha'], 
                                       reg_lambda=param['reg_lambda'],
                                       subsample= 0.7, colsample_bytree = 0.7, objective = 'reg:squarederror'))
    
best_model, best_model_index, df_performance = train_xgboost_model(k_folds_data, models)
df_performance

### Best Model for All Features

The following table summarizes the performance of each model:

| Model* | Mean of training RMSLE | Mean of validation RMSLE |
| --- | --- | --- |
| Ridge Regression | 0.1027 | 0.1163 |
| Lasso Regression | 0.1029 | 0.1159 |
| ElasticNet Regression | 0.1043 | 0.1157 |
| XGBoost | 0.0532 | 0.1182 |
<br>

\* Best hyperparameter(s) for each model is listed below. <br>
Ridge Regression -> alpha = 30; <br>
Lasso Regression -> alpha = 0.001; <br>
ElasticNet Regression -> alpha = 0.03; l1_ratio = 0.03; <br>
XGBoost -> n_estimator = 3080; max_depth = 5; learning_rate = 0.003; reg_alpha = 0.051221; reg_lambda = 0.058402; <br>


As shown in the table, all the machine learning models has very similar validation performance, with ElasticNet regression has the best validation performance.

## Feature Selection<a class="anchor" id="feature-selection"></a>

After trying to train the machine learning models with all the features, a subset of features will be selected to train the machine learning models. Based on the results of exploratory data analysis, several features were found to be worth dropping. Features that can be dropped because of their high correlation with another feature are GarageYrBuilt, TotRmsAbvGrd, GarageCars and TotalBsmtSF. Whereas, features that can be dropped because of their low correlation with SalePrice are ScreenPorch, MoSold, LowQualFinSF, 3SsnPorch, MiscVal, PoolArea, BsmtFinSF2, YrSold, BsmtHalfBath and Utilities. Two different sets of features will be attempted, and they are:

1) All features excluding highly correlated features <br>
2) All features excluding highly correlated features and features with low correlation with SalePrice

### Set 1

All features excluding highly correlated features (GarageYrBuilt, TotRmsAbvGrd, GarageCars and TotalBsmtSF)

In [ ]:
# Define URL of training set
dirname = '/kaggle/input'
subdirname = 'dataset'
train_filename = 'train_clean_EDA.csv'
train_filepath = os.path.join(dirname, subdirname, train_filename)

# Define number of folds for cross validation
n_folds = 5

# Define variables to be dropped for set 1
unwanted_vars_dict = {'dis': ['GarageYrBlt', 'TotRmsAbvGrd', 'GarageCars'],
                      'con': ['TotalBsmtSF']
                     }

# Define URL of preprocessed data file 
dirname = '/kaggle/input/dataset'
k_folds_data_filename = 'k_folds_data_set_1.pkl'
k_folds_data_filepath = os.path.join(dirname, k_folds_data_filename)

if os.path.exists(k_folds_data_filepath):
    # If file exists, then load data from the file
    with open(k_folds_data_filepath, 'rb') as f:
        k_folds_data_set1 = pickle.load(f)
else:
    # Else, store the data in a file to avoid re-computation in the future
    with open(os.path.join('/kaggle/working', k_folds_data_filename ), 'wb') as f:
        k_folds_data_set1 = data_preparation(train_filepath, n_folds, unwanted_vars_dict)
        pickle.dump(k_folds_data_set1, f)

### Ridge Regression

In [ ]:
# Parameter grid for ridge regression model
param_grid = {'alpha':[0, 1e-1, 3e-1, 1, 3, 10, 30, 100, 300, 1000]}
param_list = list(ParameterGrid(param_grid))

# Train ridge regression model
models = [Ridge(alpha=val) for param in param_list for _, val in param.items() ]
best_model_index, df_performance = train_model(k_folds_data_set1, models)
df_performance

### Lasso Regression

In [ ]:
# Parameter grid for lasso regression model
param_grid = {'alpha':[0, 1e-5, 3e-5, 1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2, 1e-1, 3e-1, 1, 3]}
param_list = list(ParameterGrid(param_grid))

# Train lasso regression model
models = [Lasso(alpha=val, max_iter=1e7) for param in param_list for _, val in param.items() ]
best_model_index, df_performance = train_model(k_folds_data_set1, models)
df_performance

### ElasticNet Regression

In [ ]:
# Parameter grid for elastic net regression model
param_grid = {'alpha':[1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2, 1e-1, 3e-1, 1],             
              'l1_ratio':[0, 1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2, 1e-1, 3e-1, 1]}
param_list = list(ParameterSampler(param_grid, n_iter=30))

# Train elastic net regression model
models = []
for param in param_list:
    models.append(ElasticNet(alpha=param['alpha'], l1_ratio=param['l1_ratio'], max_iter=1e7))
best_model_index, df_performance = train_model(k_folds_data_set1, models)
df_performance

### eXtreme Gradient Boosting (XGBoost)

In [ ]:
# Parameter grid for XGBoost
param_grid = {'n_estimator':list(range(1500, 3500, 20)), 
              'max_depth':[3, 4, 5, 6, 7],
              'learning_rate':[0.001, 0.003, 0.01, 0.03, 0.1, 0.3],
              'reg_alpha':uniform(),
              'reg_lambda':uniform(),
             }
param_list = list(ParameterSampler(param_grid, n_iter=200))


# Train XGBoost model
models = []
for param in param_list:
    models.append(xgboost.XGBRegressor(n_estimators=param['n_estimator'], 
                                       max_depth=param['max_depth'], 
                                       learning_rate=param['learning_rate'], 
                                       reg_alpha=param['reg_alpha'], 
                                       reg_lambda=param['reg_lambda'],
                                       subsample= 0.7, colsample_bytree = 0.7, objective = 'reg:squarederror'))
    
best_model, best_model_index, df_performance = train_xgboost_model(k_folds_data_set1, models)
df_performance

### Best Model for Set 1 Features

The following table summarizes the performance of each model for set 1 features:

| Model* | Mean of training RMSLE | Mean of validation RMSLE |
| --- | --- | --- |
| Ridge Regression | 0.1021 | 0.1164 |
| Lasso Regression | 0.1033 | 0.1161 |
| ElasticNet Regression | 0.1041 | 0.1160 |
| XGBoost | 0.0634 | 0.1191 |
<br>

\* Best hyperparameter(s) for each model is listed below. <br>
Ridge Regression -> alpha = 10; <br>
Lasso Regression -> alpha = 0.001; <br>
ElasticNet Regression -> alpha = 0.01; l1_ratio = 0.1; <br>
XGBoost -> n_estimator = 1500; max_depth = 4; learning_rate = 0.03; reg_alpha = 0.003169; reg_lambda = 0.108066; <br>

As shown in the table, all the machine learning models have slightly worse performances when highly correlated features are removed (set 1) as compared to when all features are used. Same as before, ElasticNet regression has the best validation performance among all the models.

### Set 2

All features excluding highly correlated features (GarageYrBuilt, TotRmsAbvGrd, GarageCars and TotalBsmtSF) and features with low correlation with SalePrice (ScreenPorch, MoSold, LowQualFinSF, 3SsnPorch, MiscVal, PoolArea, BsmtFinSF2, YrSold, BsmtHalfBath and Utilities)

In [ ]:
# Define URL of training set
dirname = '/kaggle/input'
subdirname = 'dataset'
train_filename = 'train_clean_EDA.csv'
train_filepath = os.path.join(dirname, subdirname, train_filename)

# Define number of folds for cross validation
n_folds = 5

# Define variables to be dropped for set 2
unwanted_vars_dict = {'nom': ['Utilities'],
                      'dis': ['GarageYrBlt', 'TotRmsAbvGrd', 'GarageCars', 'MoSold', 'YrSold', 'BsmtHalfBath'],
                      'con': ['TotalBsmtSF', 'ScreenPorch', 'LowQualFinSF', '3SsnPorch', 'MiscVal', 'PoolArea', 'BsmtFinSF2']
                     }

# Define URL of preprocessed data file 
dirname = '/kaggle/input/dataset'
k_folds_data_filename = 'k_folds_data_set_2.pkl'
k_folds_data_filepath = os.path.join(dirname, k_folds_data_filename)

if os.path.exists(k_folds_data_filepath):
    # If file exists, then load data from the file
    with open(k_folds_data_filepath, 'rb') as f:
        k_folds_data_set2 = pickle.load(f)
else:
    # Else, store the data in a file to avoid re-computation in the future
    with open(os.path.join('/kaggle/working', k_folds_data_filename ), 'wb') as f:
        k_folds_data_set2 = data_preparation(train_filepath, n_folds, unwanted_vars_dict)
        pickle.dump(k_folds_data_set2, f)

### Lasso Regression

In [ ]:
# Parameter grid for lasso regression model
param_grid = {'alpha':[1e-5, 3e-5, 1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2, 1e-1, 3e-1, 1, 3]}
param_list = list(ParameterGrid(param_grid))

# Train lasso regression model
models = [Lasso(alpha=val, max_iter=1e7) for param in param_list for _, val in param.items() ]
best_model_index, df_performance = train_model(k_folds_data_set2, models)
df_performance

### Ridge Regression

In [ ]:
# Parameter grid for ridge regression model
param_grid = {'alpha':[0, 1e-1, 3e-1, 1, 3, 10, 30, 100, 300, 1000]}
param_list = list(ParameterGrid(param_grid))

# Train ridge regression model
models = [Ridge(alpha=val) for param in param_list for _, val in param.items() ]
best_model_index, df_performance = train_model(k_folds_data_set2, models)
df_performance

### ElasticNet Regression

In [ ]:
# Parameter grid for elastic net regression model
param_grid = {'alpha':[1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2, 1e-1, 3e-1, 1],             
              'l1_ratio':[1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2, 1e-1, 3e-1, 1]}
param_list = list(ParameterSampler(param_grid, n_iter=30))

# Train elastic net regression model
models = []
for param in param_list:
    models.append(ElasticNet(alpha=param['alpha'], l1_ratio=param['l1_ratio'], max_iter=1e7))
best_model_index, df_performance = train_model(k_folds_data_set2, models)
df_performance

### eXtreme Gradient Boosting (XGBoost)

In [ ]:
# Parameter grid for XGBoost
param_grid = {'n_estimator':list(range(1500, 3500, 20)), 
              'max_depth':[3, 4, 5, 6, 7],
              'learning_rate':[0.001, 0.003, 0.01, 0.03, 0.1, 0.3],
              'reg_alpha':uniform(),
              'reg_lambda':uniform(),
             }
param_list = list(ParameterSampler(param_grid, n_iter=200))


# Train XGBoost model
models = []
for param in param_list:
    models.append(xgboost.XGBRegressor(n_estimators=param['n_estimator'], 
                                       max_depth=param['max_depth'], 
                                       learning_rate=param['learning_rate'], 
                                       reg_alpha=param['reg_alpha'], 
                                       reg_lambda=param['reg_lambda'],
                                       subsample= 0.7, colsample_bytree = 0.7, objective = 'reg:squarederror'))
    
best_model, best_model_index, df_performance = train_xgboost_model(k_folds_data_set2, models)
df_performance

### Best Model for Set 2 Features

The following table summarizes the performance of each model for set 2 features:

| Model* | Mean of training RMSLE | Mean of validation RMSLE |
| --- | --- | --- |
| Ridge Regression | 0.1038 | 0.1161 |
| Lasso Regression | 0.1049 | 0.1162 |
| ElasticNet Regression | 0.1040 | 0.1161 |
| XGBoost | 0.0357 | 0.1197 |
<br>

\* Best hyperparameter(s) for each model is listed below. <br>
Ridge Regression -> alpha = 10; <br>
Lasso Regression -> alpha = 0.001; <br>
ElasticNet Regression -> alpha = 0.01 ; l1_ratio = 0.0003; <br>
XGBoost -> n_estimator = 2460; max_depth = 7; learning_rate = 0.01; reg_alpha = 0.045753; reg_lambda = 0.305358; <br>

As shown in the table, all the machine learning models, expect ridge regression, have slightly worse performances when highly correlated features and features with high correlation with SalePrice (set 2) are removed, as compared to when all features or set 1 features are used. On the other hand, ridge regression has the best performance when set 2 features are used, as compared to when all features or set 1 features are used
ElasticNet regression and Ridge regression have the same validation performance, which is the best among all the models

## Summary<a class="anchor" id="summary"></a>

To encode categorical variables, ordinal encoding was used for encoding ordinal variables, whereas one-hot encoding was used for encoding nominal variables with small cardinality and that seem to have substantial impact of house price (in order to retain all the information). For nominal variables with high cardinality, hash encoding was used. After encoding was completed, standardization was performed on all the features. The final preprocessing step was to use k-nearest neighbors (KNN) imputer to impute missing values.

It was found that the best model performance was obtained when all features were used, as compared to when set 1 and set 2 features were used. Using 5-folds cross validation, the best model found in this project is ElasticNet regression (alpha = 0.03; l1_ratio =0.03), which has a validation root mean square log error (RMSLE) of 0.1157.

